# Gesture Volume Control Project

        The script is a "Gesture Volume Control" program that uses hand gestures captured by a camera to control the volume of the speakers. It utilizes the OpenCV library to process video frames and the handLMmodule, a custom module, to detect hands within the frames. The position of the hand is then used to control the volume of the speakers using the pycaw library

Here import Handlandmarks module

In [ ]:
import time
import cv2
import numpy as  np 
import handLMmodule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume


In [ ]:
######################
wCam, hCam = 640, 480
######################

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4,hCam)
ptime = 0
detector = htm.handDetector(detectionCon=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

inds the position of the hand, and uses the position to control the volume of the speakers using the pycaw library. It also displays a volume bar and percentage on the screen, and a circle on the center of the hand to indicate the volume level. 

In [ ]:
while True:
    success, img=cap.read()
    img = detector.findHands(img)
    lmlist = detector.findPosition(img, draw=False)
    if len(lmlist) !=0:    
        # print(lmlist[4], lmlist[8])

        x1, y1 = lmlist[4][1], lmlist[4][2]
        x2, y2 = lmlist[8][1], lmlist[8][2]
        cx, cy = (x1+x2) // 2, (y1+y2) // 2

        
        cv2.circle(img, (x1,y1), 10,(255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2,y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        # print(length)

        # Hand Range 30 to 200
        # volume range -65 to 0
        vol = np.interp(length, [30, 250], [minVol, maxVol])
        volBar = np.interp(length, [30, 250], [400, 150])
        volPer = np.interp(length, [30, 250], [0, 100])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)

        if length <=50:
            cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)

    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, f'per:{(int(volPer))}%', (90, 410), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0,  0), 2)      


In [ ]:
   ctime = time.time()
    fps = 1 / (ctime-ptime)
    ptime = ctime
    # cv2.putText(img, f'fps:{(int(fps))}', (70, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    cv2.imshow('image', img)
    cv2.waitKey(1)